In [ ]:
# 파인튜닝된 LLaMA3 모델을 불러와 뉴스 문장을 어린이용 문장으로 변환

# 필요 패키지

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 추론

In [ ]:
MODEL_PATH = "./checkpoints/llama3_finetuned"

# 시스템 프롬프트 설정
SYSTEM_PROMPT = "당신은 유용한 AI 어시스턴트입니다. 사용자의 요구사항에 맞게 문장 변환을 해야합니다."

def load_model():
    """파인튜닝된 모델과 토크나이저 로드"""
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        device_map="auto",
        torch_dtype=torch.float16
    )
    return model, tokenizer

def translate(text, model, tokenizer):
    """주어진 뉴스 문장을 어린이 뉴스로 변환"""
    instruction = f"'{text}'라는 문장을 어린이가 이해하기 쉽게 구어체로 바꿔줘."
    prompt = f"<system>{SYSTEM_PROMPT}</system>\n<user>{instruction}</user>\n<assistant>"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.1
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    # 프롬프트 제거 후 결과만 추출
    generated = decoded.replace(prompt, "").strip()
    if "</assistant>" in generated:
        generated = generated.split("</assistant>")[0].strip()
    return generated

if __name__ == "__main__":
    model, tokenizer = load_model()

    # 테스트 문장
    input_text = "윤석열 대통령이 김영선 전 의원 공천을 직접 지시했다는 녹음이 공개됐습니다."

    result = translate(input_text, model, tokenizer)

    print("\n[원문]")
    print(input_text)
    print("\n[어린이용 뉴스]")
    print(result)